In [2]:
#packages to be used
from satpy import Scene, find_files_and_readers
from pyresample import create_area_def
from satpy.writers import get_enhanced_image
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from celluloid import Camera

from satpy import MultiScene
from satpy.multiscene import timeseries

In [7]:
# Animation

# Generates a list of file names, sorted alphabetically
filenames = glob('/Users/ericmorales/Desktop/RS_Files/HC_Delta_TimeSeries/MYD021KM.*')
filenames.sort()
filenames

# 'clip' the day identifier (e.g. 'A2022060') from all file names. This establishes the seperate days that we 
#     want to make into an animation
swats = np.unique([f.split('.')[1] for f in filenames])
swats

# set extent
extent=[-94, 27.5, -88, 30.5]

#create area once
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5},
                          width=1500, height=750,
                          area_extent=extent, units='degrees')
#constants. This is for an outdated, significantly simpler version of the CDOM algorithm we now use
A = 0.472; B = 1.48; C = 4.64

# establishing a figure for the animation
fig =  plt.figure(figsize=(6, 4), dpi=400)
camera = Camera(fig)

#The loop we'll use. Each iteration makes one frame of the animation.
for swat in swats:
    # groups the files for each day, so that we can plot them together
    dayfiles = glob(base+swat+'*')
    # establishes the reader we'll use. This will change depending on what satellite file you're using
    scn = Scene(dayfiles, reader='modis_l1b')
    # loading channels for CDOM calculation
    scn.load(['10','4'])
    # loading true color channel
    scn.load(['true_color'])

    #reproject
    new_scn = scn.resample(my_area)

    #generate RGB from true color
    rgb = get_enhanced_image(new_scn['true_color'])

    #calculate aCDOM
    aCDOM412 = (np.log((new_scn['10']/new_scn['4'] - A)/ B))/(-C)
    aCDOM412 = aCDOM412.compute()

    #extract projection and lon lat from products
    crs = new_scn['true_color'].attrs['area'].to_cartopy_crs()
    lons, lats = new_scn['true_color'].attrs['area'].get_lonlats()

    #left true color
    ax1 = plt.subplot(1, 2, 1, projection=crs)
    rgb.data.plot.imshow(rgb='bands', transform=crs, ax=ax1)
    ax1.set_title('True Color')

    #right aCDOM
    trim = aCDOM412.max().values
    ax2 = plt.subplot(1, 2, 2, projection=crs)
    #costline with specified resolution
    ax2.coastlines(res)
    #this is ploting x,y, aCDOM
    ax2.pcolormesh(lons, lats, aCDOM412.where(aCDOM412<=trim), transform=ccrs.PlateCarree(),
                  vmin=.01, vmax=.3)
    #mask the land
    ax2.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                                name='land', facecolor='grey',
                                                scale=res))
    ax2.set_title('aCDOM412')
    #optimize spacing between plots
    fig.tight_layout()
    
    #for each frame of animation
    camera.snap()
    
    #save. The interval determines the frame rate in miliseconds.
animation = camera.animate(interval=1000)
animation.save('animation.mp4')

<Figure size 2400x1600 with 0 Axes>